# Runpod Dev Environment Setup

SSH implements public-key cryptography to establish trust between client and server systems. In this paradigm, we have the **public key** (`.pub`) that functions as an access *verifier*, installed on target systems (like Runpod nodes), and the **private key** serves as the unique authentication factor for mathematically proving identity.

Below, we create public keys which we install onto Runpod and GitHub systems. The corresponding private key for Runpod is stored locally, while that for GitHub is securely copied via `scp`. This gives every new pod read and write access to GitHub repositories.

![](./img/runpod-ssh.png)

## Give local SSH access to pods

1. Generate SSH keys for Runpod (no passphrase):
```{.bash filename="$ (local)"}
ssh-keygen -t ed25519 -C "runpod" -f ~/.ssh/runpod -N ""
cat ~/.ssh/runpod.pub
```
2. Add this to **Settings**> **SSH Public Keys** in Runpod (separated by newline).

## Give pods SSH access to GitHub
1. Generate SSH keys to access GitHub from each pod (no passphrase):
```{.bash filename="$ (local)"}
ssh-keygen -t ed25519 -C "runpod-github" -f ~/.ssh/runpod_github -N ""
cat ~/.ssh/runpod_github.pub
```
2. Add the public SSH key to your GitHub account under **Settings**> **SSH and GPG Keys**:

![](./img/tooling-github-ssh.png)

3. Run the following commands. Copy your pod's **SSH over exposed TCP** [connection string]{.underline}[^connstring] and paste it in the resulting input prompt. You should verify the auto-filled values for **pod IP** and **port number** before proceeding.
```{.bash filename="$ (local)"}
printf "SSH over exposed TCP connection string:\n"
read RUNPOD_SSH
RUNPOD_IP=$(echo "$RUNPOD_SSH" | sed -E 's/.*@([0-9.]+).*/\1/') # <1>
RUNPOD_PORT=$(echo "$RUNPOD_SSH" | sed -E 's/.*-p ([0-9]+).*/\1/') # <2>
```
1. Extract IP as number or `.` after `@` until space or end.
2. Extract port as number after `-p` flag until space or end.

<!-- sdfsdfsd -->

4. The following logs into the pod and ensures **access to GitHub** repos:
```{.bash filename="$ (local)"}
scp -P $RUNPOD_PORT -i ~/.ssh/runpod -o StrictHostKeyChecking=no \
    ~/.ssh/runpod_github root@$RUNPOD_IP:/root/.ssh/id_rsa  # <0>

ssh -t -p $RUNPOD_PORT -i ~/.ssh/runpod root@$RUNPOD_IP '
    chmod 600 ~/.ssh/id_rsa # <1>
    ssh-keyscan github.com >> ~/.ssh/known_hosts 2>/dev/null    # <2>
    cd ~ && exec bash -l    # <3>
'
```
0. Copies GitHub private key to pod to `.ssh/id_rsa` so it's automatically detected.
1. Sets permissions so only owner can read or write on the private key file.
2. Adds scanned pod's public SSH hostkey to local `known_hosts` to skip authenticity prompts.
3. Changes working directory to home (`~`, i.e. `/root`) and login to pod. The `-t` flag on the `ssh` command allows `exec` to work interactively.

[^connstring]: e.g. `ssh root@63.141.33.33 -p 22011 -i ~/.ssh/id_ed25519`

:::{.callout-tip}
Collecting the commands in a single script ([-@lst-runpod]) that you can run to connect to any new pod:
```{.bash filename="$ (local)"}
chmod +x ./scripts/runpod.sh
./scripts/runpod.sh
```
:::


## Virtual environment

Inside the pod, we can now clone repositories. For example:

```{.bash filename="$ (root@runpod)"}
git clone git@github.com:particle1331/ai-notebooks.git
```

Our recommended approach is to use `uv` to build a venv at `.venv` that is synced using `uv.lock`. This environment can then be used as Jupyter kernel [in vscode](https://code.visualstudio.com/docs/datascience/jupyter-kernel-management). Or run scripts using [`uv run`](https://docs.astral.sh/uv/concepts/projects/run/#running-scripts). The required Python version is also specified ([-@lst-Makefile]):

```{.bash filename="$ (root@runpod)"}
make venv
```

Alternatively, you can use `pip` to install packages (without using `uv`):
```{.bash filename="$ (root@runpod)"}
# install required python version, activate venv
make uv
uv python install 3.13
uv venv .venv && source .venv/bin/activate
curl -sS https://bootstrap.pypa.io/get-pip.py | .venv/bin/python

# install requirements on venv
make requirements
pip install -r requirements.txt
pip install -e .
```

:::{.callout-note}
Some environments, like AzureML where compute instances have network-mounted or ephemeral filesystems, can cause `uv` failures. In the case of short-lived pods, you don't necessarily need to perfectly setup an environment or maintain a clean state. On the other hand, reproducibility benefits from a controlled, well-defined environment. As usual, the level of precision needed depends on the scope of the project.
:::

## Quarto docs

Check [here](https://github.com/particle1331/ai-notebooks/blob/main/.github/workflows/publish.yml) for the Quatro version that we're using. Adjust the following variable accordingly:
```{.bash filename="$ (root@runpod)"}
export QUARTO_VERSION=1.7.32  # may be outdated
wget -q https://github.com/quarto-dev/quarto-cli/releases/download/v${QUARTO_VERSION}/quarto-${QUARTO_VERSION}-linux-amd64.deb
dpkg -i quarto-${QUARTO_VERSION}-linux-amd64.deb && rm quarto-${QUARTO_VERSION}-linux-amd64.deb
```

Preview then [port-forward](https://code.visualstudio.com/docs/remote/ssh#_forwarding-a-port-creating-ssh-tunnel) using vscode:
```{.bash filename="$ (root@runpod)"}
make docs
```

## Appendix: Tmux

Tmux allows you to open multiple windows in a **single SSH session** to a remote server, without needing to log in separately for each window. You can also detach from a tmux session and reconnect later to resume your work exactly where you left off, without having to log in again or restart your processes. This means you can run several tasks in parallel, switch between them easily, and keep them running even if your connection drops.

For convenience, I use [byobu](https://www.byobu.org/downloads). In this section, I'll list commands and workflows I found useful.

| Command | Function |
| :--: | :-- |
| F1, Shift + F1 | Display help |
| Ctrl + F2 | Create new split vertically |
| Shift + F2 | Create new split horizontally |
| F2 | Create new window |
| F8 | Rename the current window |
| Ctrl + F6 | Kill a focused split |
| F3 / F4 | Switch between windows |
| Shift + F3 / F4 | Switch between splits |
| F6 | Detach session |
| Shift + F9 | Run command on all splits. See @fig-byobu-command-splits. |
: Byobu commands {tbl-colwidths="[30,70]"}

<br>

Running the same command on multiple splits (very useful):

![Running a dynamic command on 3 splits.](./img/byobu-commands.png){#fig-byobu-command-splits}

**Resuming.** Working on a remote server, you might lose your connection unexpectedly or intentionally disconnect while wanting to keep processes running. In such cases, you can **press F6** in Byobu to detach the session and logout. Once you SSH again to the server, the session will be restored by running `byobu`. This is very useful!

![Resuming a detached session. The process kept running in the background while we were away.](./img/byobu-detached.png){#fig-byobu-command-splits}

## Appendix: Code listings

::: {#lst-Makefile lst-cap="Makefile for the project."}
```{.bash filename=Makefile}
{{< include "../../Makefile" >}}
```
:::

::: {#lst-runpod lst-cap="Script to SSH to Runpod instance."}
```{.bash filename="./scripts/runpod.sh"}
{{< include "../../scripts/runpod.sh" >}}
```
:::